<a href="https://colab.research.google.com/github/Heinkek-99/11/blob/master/GEMA_DA_Maching_Learning_devoir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Simulation de données

[texte du lien](https://)Simuler la génération de y.

$$
y = 5*sin(\frac{3}{7}*x)+ \epsilon, \quad \epsilon \sim \mathcal{N}(0, \frac{1}{4})
$$

#### Méthode de kfold

En utilisation la méthode k-fold de validation croisée et une méthode de régression polynômiale, réalisez un modèle de regression.

#### Arbre de décision : donnée Titanic

Dans ce jeu de donnée, supprimer les données. L'objectif est de construire un modèle pour prédire ceux qui ont survécu au naufrage ou pas. La variable cible est survie.

#### Résumé en un paragraphe votre compréhension du compromis biais-variance

In [ ]:
# %% [markdown]
# # Projet de Modélisation - Compromis Biais-Variance

# %% [markdown]
# ## Partie 1 : Régression Polynomiale avec Validation Croisée

# %% [markdown]
# ### Équation à modéliser :
# $$ y = 5\sin\left(\frac{3}{7}x\right) + \epsilon,\quad \epsilon \sim \mathcal{N}(0, 0.25) $$

# %%
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Génération des données
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 5 * np.sin((3/7)*x)
y = y_true + np.random.normal(0, 0.5, size=len(x))

plt.figure(figsize=(10, 5))
plt.scatter(x, y, s=10, label='Données avec bruit')
plt.plot(x, y_true, c='red', label='Vraie relation')
plt.legend()
plt.title('Génération des données synthétiques')
plt.show()

# %%
# Validation croisée k-fold
k = 5
degrees = range(1, 11)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

mse_train = []
mse_val = []

for degree in degrees:
    mse_train_fold = []
    mse_val_fold = []

    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(x.reshape(-1, 1))

    for train_idx, val_idx in kf.split(X_poly):
        X_train, X_val = X_poly[train_idx], X_poly[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = LinearRegression()
        model.fit(X_train, y_train)

        mse_train_fold.append(mean_squared_error(y_train, model.predict(X_train)))
        mse_val_fold.append(mean_squared_error(y_val, model.predict(X_val)))

    mse_train.append(np.mean(mse_train_fold))
    mse_val.append(np.mean(mse_val_fold))

# %%
# Visualisation des résultats
plt.figure(figsize=(10, 5))
plt.plot(degrees, mse_train, 'o-', label='Erreur entraînement')
plt.plot(degrees, mse_val, 'o-', label='Erreur validation')
plt.xlabel('Degré polynomial')
plt.ylabel('MSE')
plt.xticks(degrees)
plt.legend()
plt.title('Courbe de complexité modèle')
plt.show()

# %%
# Meilleur modèle
best_degree = degrees[np.argmin(mse_val)]
print(f"Degré optimal : {best_degree}")

poly = PolynomialFeatures(degree=best_degree)
X_poly = poly.fit_transform(x.reshape(-1, 1))
model = LinearRegression().fit(X_poly, y)

x_plot = np.linspace(0, 10, 500)
X_plot = poly.transform(x_plot.reshape(-1, 1))
y_pred = model.predict(X_plot)

plt.figure(figsize=(10, 5))
plt.scatter(x, y, s=10, label='Données')
plt.plot(x_plot, y_pred, c='red', label='Modèle optimal')
plt.plot(x, y_true, '--', c='green', label='Vraie relation')
plt.legend()
plt.title(f'Régression polynomiale degré {best_degree}')
plt.show()

# %% [markdown]
# ## Partie 2 : Arbre de Décision sur les données du Titanic

# %%
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# Chargement des données
titanic = pd.read_csv('Titanic - Titanic.csv')

# Prétraitement
titanic_clean = titanic.drop(columns=['nom'])
titanic_clean['sexe'] = titanic_clean['sexe'].map({1: 0, 2: 1})  # Conversion en binaire
titanic_clean['age'].fillna(titanic_clean['age'].median(), inplace=True)

# Séparation des données
X = titanic_clean[['classe', 'sexe', 'age', 'tarif']]
y = titanic_clean['survie']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# %%
# Entraînement du modèle
tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

# Évaluation
y_pred = tree.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion')
plt.show()

# %%
# Importance des caractéristiques
feature_importance = pd.Series(tree.feature_importances_, index=X.columns)
feature_importance.sort_values().plot(kind='barh')
plt.title('Importance des caractéristiques')
plt.show()

# %% [markdown]
# ## Partie 3 : Compromis Biais-Variance

# %% [markdown]
# **Analyse** :
# - **Biais** : Erreur systématique due à des hypothèses simplificatrices (ex: modèle linéaire pour des données non-linéaires).
# - **Variance** : Sensibilité aux fluctuations des données d'entraînement (ex: modèle trop complexe mémorise le bruit).
#
# Dans la régression polynomiale :
# - Degré 1 : Biais élevé (sous-ajustement)
# - Degré 10 : Variance élevée (surajustement)
#
# Pour le Titanic :
# - Arbre profond : Variance élevée (règles trop spécifiques)
# - Arbre peu profond : Biais élevé (règles trop générales)
#
# L'objectif est de trouver le point optimal où la somme biais² + variance est minimale, ce que permettent les techniques de validation croisée et de régularisation.